In [20]:
import folium
import os
import webbrowser
import sys
import seaborn as sns
from datetime import datetime
import utils
import pandas as pd
import glob
import numpy as np
import ipywidgets
from ipywidgets import interactive, widgets, Checkbox
from IPython.display import display, clear_output
from geopy import distance
from multiprocessing import Pool
from research_2 import *

In [2]:
path_to_data = ('../data/0626-0709_MERGED.csv')

df = pd.read_csv(path_to_data)
# Convert 'start_time' to datetime and extract date only
df['start_time'] = pd.to_datetime(df['start_time'])
df['date'] = df['start_time'].dt.date

# Get unique IDs and Dates
unique_ids = sorted(df['id'].unique())
unique_dates = sorted(df['date'].unique())

# Create dropdowns
id_dropdown = widgets.Dropdown(options = unique_ids, description='ID:')
# Create SelectMultiple for dates
date_dropdown = widgets.SelectMultiple(options = unique_dates, description='Date:', value=[unique_dates[0]])
# Slider widget
duration_slider = widgets.IntSlider(
    min=0,
    max=7200,
    step=900,
    description='Duration greater than:',
    value=0
)
# Create the checkbox
clustered_checkbox = Checkbox(value=False, description='Use clustered markers')
# Create the button
button = widgets.Button(description='Generate')

def on_button_click(b):
    render_consined_map_multi_days(path_to_data, id_dropdown.value, 1.0, date_dropdown.value, duration_slider.value, clustered_checkbox.value)

# Set the callback function to the button
button.on_click(on_button_click)

# Display dropdowns
display(id_dropdown)
display(date_dropdown)
display(clustered_checkbox)
display(duration_slider)
display(button)

Dropdown(description='ID:', options=(454000000000001, 510010042243704, 510010042641098, 510010064800584, 51001…

SelectMultiple(description='Date:', index=(0,), options=(datetime.date(2023, 6, 26), datetime.date(2023, 6, 27…

Checkbox(value=False, description='Use clustered markers')

IntSlider(value=0, description='Duration greater than:', max=7200, step=900)

Button(description='Generate', style=ButtonStyle())

510010042243704
POINT COUNT 737


In [21]:
df_hw_1 = pd.read_csv('../data/HW_0622.csv')
df_hw_1

,id,home_lat,home_lon,work_lat,work_lon,date
0,454000000000001,-7.285050,112.808345,-6.197165,106.826833,2023-06-22
1,510010042243704,-6.266572,106.876891,-6.261372,106.875643,2023-06-22
2,510010042641098,-7.403045,110.534783,-7.403045,110.534783,2023-06-22
3,510010064800584,-7.560058,109.041864,-7.517071,109.117061,2023-06-22
4,510010065014725,-6.173856,106.891388,-6.174718,106.888255,2023-06-22
...,...,...,...,...,...,...
347,510019945126427,-7.330828,110.120570,-7.297793,110.169925,2023-06-22
348,510090000000000,-7.412484,112.718656,-7.955417,112.029419,2023-06-22
349,510092038161045,-6.908243,107.578037,-6.903552,107.565301,2023-06-22
350,510093019800295,-7.950570,112.698249,-7.950570,112.698249,2023-06-22


## DEV-TESTS

In [15]:
def add_suffixes(df, suffix):
    df.columns = [f"{col}{suffix}" if col != 'id' else col for col in df.columns]
    return df

df_hw_1 = pd.read_csv('../data/HW_0622.csv')
df_hw_2 = pd.read_csv('../data/HW_0702.csv')
df_hw_3 = pd.read_csv('../data/HW_0709.csv')

df_hw_1 = add_suffixes(df_hw_1, '_x')
df_hw_2 = add_suffixes(df_hw_2, '_y')
df_hw_3 = add_suffixes(df_hw_3, '_z')

df_hw_12 = df_hw_1.merge(df_hw_2, on='id', how='inner')
df_hw_all = df_hw_12.merge(df_hw_3, on='id', how='inner')

# select the float columns
float_cols = [c for c in df_hw_all if df_hw_all[c].dtype == "float64"]
# round the float columns to 5 decimal places
df_hw_all[float_cols] = df_hw_all[float_cols].round(7)

df_duped_home = df_hw_all[(df_hw_all['home_lat_x'] == df_hw_all['home_lat_y']) & 
                          (df_hw_all['home_lat_x'] == df_hw_all['home_lat_z']) &
                          (df_hw_all['home_lon_x'] == df_hw_all['home_lon_y']) &
                          (df_hw_all['home_lon_x'] == df_hw_all['home_lon_z'])]

df_duped_work = df_hw_all[(df_hw_all['work_lat_x'] == df_hw_all['work_lat_y']) & 
                          (df_hw_all['work_lat_x'] == df_hw_all['work_lat_z']) &
                          (df_hw_all['work_lon_x'] == df_hw_all['work_lon_y']) &
                          (df_hw_all['work_lon_x'] == df_hw_all['work_lon_z'])]
df_duped_work

,id,home_lat_x,home_lon_x,work_lat_x,work_lon_x,date_x,home_lat_y,home_lon_y,work_lat_y,work_lon_y,date_y,home_lat_z,home_lon_z,work_lat_z,work_lon_z,date_z
1,510010042243704,-6.266572,106.876891,-6.261372,106.875643,2023-06-22,-6.266572,106.876891,-6.261372,106.875643,2023-07-02,-6.266572,106.876891,-6.261372,106.875643,2023-07-09
2,510010042641098,-7.403045,110.534783,-7.403045,110.534783,2023-06-22,-7.403045,110.534783,-7.403045,110.534783,2023-07-02,-7.403045,110.534783,-7.403045,110.534783,2023-07-09
7,510010143982819,-7.399009,110.193077,-7.399009,110.193077,2023-06-22,-7.399009,110.193077,-7.399009,110.193077,2023-07-02,-7.399009,110.193077,-7.399009,110.193077,2023-07-09
9,510010144567399,-7.232193,110.351543,-7.227473,110.358586,2023-06-22,-7.232193,110.351543,-7.227473,110.358586,2023-07-02,-7.232193,110.351543,-7.227473,110.358586,2023-07-09
11,510010164132856,-7.442711,111.075583,-7.442711,111.075583,2023-06-22,-7.442711,111.075583,-7.442711,111.075583,2023-07-02,-7.442711,111.075583,-7.442711,111.075583,2023-07-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,510019533342184,-7.461875,112.389205,-7.470100,112.370307,2023-06-22,-7.461875,112.389205,-7.470100,112.370307,2023-07-02,-7.473332,112.377387,-7.470100,112.370307,2023-07-09
286,510019667446943,0.466037,121.099168,0.457900,121.102232,2023-06-22,0.457900,121.102232,0.457900,121.102232,2023-07-02,0.457900,121.102232,0.457900,121.102232,2023-07-09
289,510019767679056,-6.171896,106.757272,-6.171896,106.757272,2023-06-22,-6.174068,106.759465,-6.171896,106.757272,2023-07-02,-6.174068,106.759465,-6.171896,106.757272,2023-07-09
296,510093019800295,-7.950571,112.698249,-7.950571,112.698249,2023-06-22,-7.950571,112.698249,-7.950571,112.698249,2023-07-02,-7.950571,112.698249,-7.950571,112.698249,2023-07-09


In [11]:
mappy = render_cosined_map_choice('../data/0626-0702_MERGED.csv', '2023-06-26', '510010431954182', 1.0)
mappy

# example: render_cosined_map_choice('../data/0626-0702_MERGED.csv', '2023-06-26', '510010431954182', 1.0)

510010431954182
POINT COUNT 74
